In [ ]:
[
  {"app": "Instagram", "duration_minutes": 120},
  {"app": "YouTube", "duration_minutes": 90},
  {"app": "Chrome", "duration_minutes": 45},
  {"app": "WhatsApp", "duration_minutes": 30},
  {"app": "Slack", "duration_minutes": 25}
]


[{'app': 'Instagram', 'duration_minutes': 120},
 {'app': 'YouTube', 'duration_minutes': 90},
 {'app': 'Chrome', 'duration_minutes': 45},
 {'app': 'WhatsApp', 'duration_minutes': 30},
 {'app': 'Slack', 'duration_minutes': 25}]

In [ ]:
# 1. Imports and Setup
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import openai
import os

# --- Load dataset ---
df = pd.read_csv('/content/screentime_analysis.csv')  # Update path as needed
df['Date'] = pd.to_datetime(df['Date'])

# --- Label Encoding ---
label_encoder = LabelEncoder()
df['App_Encoded'] = label_encoder.fit_transform(df['App'])
app_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# --- Feature Engineering ---
df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Day_of_Month'] = df['Date'].dt.day

# Keep a copy for summarization
df_original = df.copy()

# --- Scaling ---
scaler = StandardScaler()
scale_cols = ['Usage (minutes)', 'Notifications', 'Times Opened', 'Day_of_Week', 'Day_of_Month']
df[scale_cols] = scaler.fit_transform(df[scale_cols])

# --- Optional: Train-test split ---
X = df[['App_Encoded', 'Notifications', 'Times Opened', 'Day_of_Week', 'Day_of_Month']]
y = df['Usage (minutes)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Create summary for LLM using original values ---
summary = (
    df_original.groupby("App")[['Usage (minutes)', 'Notifications', 'Times Opened']]
    .agg({
        'Usage (minutes)': 'sum',
        'Notifications': 'mean',
        'Times Opened': 'mean'
    })
    .sort_values("Usage (minutes)", ascending=False)
    .round(2)
)

summary_text = summary.to_string()

# --- Generate prompt ---
prompt = f"""
You are a digital wellbeing coach.

Here's the mobile app usage summary:
{summary_text}

Based on this data:
1. Identify the most time-consuming apps.
2. Suggest usage patterns based on notifications and app opens.
3. Recommend ways to improve screen time habits.
4. Offer tips for better digital balance.
"""

# --- OpenAI API Call ---
openai.api_key = os.getenv("OPENAI_API_KEY")  # Better to use environment variable

try:
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Or "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful digital wellbeing assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    # --- Display Result ---
    print("\n--- Digital Wellbeing Insights ---\n")
    print(response.choices[0].message.content)

except Exception as e:
    print(f"Error calling OpenAI API: {e}")

Error calling OpenAI API: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define list of common apps and categories
apps = ['Instagram', 'YouTube', 'WhatsApp', 'Facebook', 'Gmail', 'Slack', 'TikTok', 'Chrome', 'Spotify', 'Calendar']

# Generate 30 days of data
dates = pd.date_range(start='2024-04-01', periods=30)

# Create synthetic usage data
data = []
for date in dates:
    for app in apps:
        usage = np.random.poisson(lam=30) if app in ['Instagram', 'YouTube', 'TikTok'] else np.random.poisson(lam=10)
        notifications = np.random.poisson(lam=5)
        times_opened = np.random.poisson(lam=3)
        data.append([date, app, usage, notifications, times_opened])

# Create DataFrame
df = pd.DataFrame(data, columns=['Date', 'App', 'Usage (minutes)', 'Notifications', 'Times Opened'])

# Show the first few rows
print(df.head())

# Save to CSV if needed
# df.to_csv("screentime_analysis.csv", index=False)


        Date        App  Usage (minutes)  Notifications  Times Opened
0 2024-04-01  Instagram               28              4             3
1 2024-04-01    YouTube               25              4             2
2 2024-04-01   WhatsApp                9              4             2
3 2024-04-01   Facebook               13              3             1
4 2024-04-01      Gmail               15              9             2
